In [1]:
import json
import logging
import requests
import re
import time

In [2]:
def takeResponseAttr(attr_name, resp_text, quote = '"'):
    attr_name = attr_name+'\s?'+'.'+'\s?'+quote
    
    res= re.findall(attr_name+'[^"]+'+quote, resp_text)
    res = res[0]
    res = re.sub(attr_name,'',res)
    res = re.sub(quote+'$','',res)
    
    return res

In [3]:
def takeResponseDict(resp_text):
    #возвращает только первый словарь.
    #будут проблемы, если будет вложенный словарь
    start = resp_text.find('{')
    end = resp_text.find('}', start)
    return eval(resp_text[start: end+1])

In [4]:
def takeConvertedDict(source_dict):
    _d={}
    for k, n in source_dict.items():
        n = n.replace('\\/','/')
        if n.count('|') > 2:
            for item in n.split('|'):
                if item.count('=') > 0:
                    i, j = item.split('=')
                else:
                    i = k
                    j = item
                _d[i] = j
        else:
            _d[k] = n
        
    return _d

In [5]:
formattedStrLambda = lambda item_dict: '&'.join(map(lambda x: '{}={}'.format(x[0],x[1]), item_dict.items()))

In [6]:
logging.basicConfig(level=logging.DEBUG)
captcha_link ='https://captcha.kik.com/?id='
captcha_id = '3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA'

ses = requests.session()
res = ses.get(captcha_link+captcha_id)

ses.headers.update({'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'})
ses.headers.update({'referer': captcha_link+captcha_id, 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'\
                    ,'authority':'funcaptcha.com'})

next1 ='https://funcaptcha.com/fc/api/?onload=callbackCaptchaReady'
res1 = ses.get(next1)
#get next link
next2 = takeResponseAttr('t.src',res1.text)
res2 = ses.get(next2)
#get next link
token_dict = {'bda':'W3sia2V5IjoiYXBpX3R5cGUiLCJ2YWx1ZSI6ImpzIn0seyJrZXkiOiJwIiwidmFsdWUiOjF9L'\
                       'Hsia2V5IjoiZiIsInZhbHVlIjoiNzY1MWVkMTE5ZmRmZDhlNWQzMTA0NGYzZjk3N2ZkMDIifSx7Im'\
                       'tleSI6ImNzIiwidmFsdWUiOjF9XQ=='\
                       ,'public_key':'92A09218-4AD3-CC2B-8145-87CD0B851124'\
                       ,'site':'https://captcha.kik.com'\
                       ,'data[kik_id]':captcha_id}
            
next3 =  takeResponseAttr('var S7e', res2.text)\
        +takeResponseAttr('var Z8e', res2.text)\
        +'&'+formattedStrLambda(takeConvertedDict(token_dict))
        
res3 = ses.get(next3)
#токен получен успешно.

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): captcha.kik.com
DEBUG:urllib3.connectionpool:https://captcha.kik.com:443 "GET /?id=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): funcaptcha.com
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/api/?onload=callbackCaptchaReady HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn.funcaptcha.com
DEBUG:urllib3.connectionpool:https://cdn.funcaptcha.com:443 "GET /fc/js/ba1e486f2d32eee8fdf97ee89ec06b14/standard/funcaptcha_api.js HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/gt2/?callback=fcJSONPCallback&bda=W3sia2V5IjoiYXBpX3R5cGUiLCJ2YWx1ZSI6ImpzIn0s

In [7]:
auth_json = takeConvertedDict(takeResponseDict(res3.text))
auth_json['ps'] = captcha_id

next4 = 'https://funcaptcha.com/fc/gc/?'\
        +formattedStrLambda(takeConvertedDict(auth_json))

#res4 = ses.get(next4, data = takeConvertedDict(auth_json))

In [8]:
ses.headers.update({'referer': next4})

form_data = {'sid': auth_json.get('r')
             ,'session_token':auth_json.get('token')
             ,'render_type':'canvas'
             ,'category':'Site+URL'
             ,'action':captcha_link+captcha_id
             ,'analytics_tier':auth_json.get('at')
            }

next6 = 'https://funcaptcha.com/fc/a/'
res6 = ses.post(next6, data = takeConvertedDict(form_data))

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "POST /fc/a/ HTTP/1.1" 200 None


In [9]:
gc_data = {'sid': auth_json.get('r')
           ,'token':auth_json.get('token')
           , 'lang': 'ru'
           ,'render_type':'canvas'
           ,'analytics_tier':auth_json.get('at')
           ,'data[ps]': captcha_id
           ,'data[status]':'init'
            }


next7 = 'https://funcaptcha.com/fc/gfct/'
res7 = ses.post(next7, data = takeConvertedDict(gc_data))

auth_json['game_token'] = takeResponseAttr('"challengeID"', res7.text)

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "POST /fc/gfct/ HTTP/1.1" 200 None


In [10]:
'''
game_data = {'sid': auth_json.get('r')
            ,'session_token':auth_json.get('token')
            ,'game_token':auth_json.get('game_token')}

next8 = 'https://funcaptcha.com/fc/ekey/'
res8 = ses.post(next8, data=game_data)
print(res8.text)
'''

"\ngame_data = {'sid': auth_json.get('r')\n            ,'session_token':auth_json.get('token')\n            ,'game_token':auth_json.get('game_token')}\n\nnext8 = 'https://funcaptcha.com/fc/ekey/'\nres8 = ses.post(next8, data=game_data)\nprint(res8.text)\n"

In [11]:
game_params = {'pkey':token_dict.get('public_key')
              ,'litejs':'1'
              ,'fb_type':'2'
              ,'session':auth_json.get('token')
              ,'r':auth_json.get('r')
              ,'lang': 'ru'
              ,'ps':auth_json.get('ps')
             }

ses.headers.clear()

ses.headers.update({'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'\
                    ,'accept-encoding':'gzip, deflate, br'\
                    ,'accept-language':'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'\
                    ,'content-type':'application/x-www-form-urlencoded'\
                    ,'referer':captcha_link+captcha_id\
                    ,'upgrade-insecure-requests':'1'\
                    ,'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'
                   })


next9 = 'https://funcaptcha.com/fc/api/nojs/?'+formattedStrLambda(game_params)
res9 = ses.get(next9)

time.sleep(1)
game_data2 = {'fc-game[session_token]':takeResponseAttr('_session_token', res9.text)
              ,'fc-game[data]': takeResponseAttr('fc-game\[data\]" value', res9.text)
             }

ses.headers.update({'referer': next9\
                    ,'origin':'https//funcaptcha.com'\
                    ,'cache-control':'max-age=0'\
                    ,'content-Type': 'text/html; charset=UTF-8'})

time.sleep(1)
res10 = ses.post(next9, data = game_data2)

#next11 = 'https://funcaptcha.com/fc/gimg/d/'+
#    5825a391e6c343157.06998379/g/1155a391e6c331b13.28870210/s/2555a391e6c12c985.99884630/sid/eu-west-1/1.png
#res11 = ses.get(captcha_link+captcha_id)

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/api/nojs/?pkey=92A09218-4AD3-CC2B-8145-87CD0B851124&litejs=1&fb_type=2&session=6605a394d1bc4e228.62741596&r=eu-west-1&lang=ru&ps=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "POST /fc/api/nojs/?pkey=92A09218-4AD3-CC2B-8145-87CD0B851124&litejs=1&fb_type=2&session=6605a394d1bc4e228.62741596&r=eu-west-1&lang=ru&ps=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA HTTP/1.1" 200 None


In [12]:
print(game_data2.get('fc-game[session_token]'))
print(auth_json.get('token'))
print(auth_json.get('game_token'))

6815a394d1cae29f4.14873587
6605a394d1bc4e228.62741596
825a394d1c5fc769.47718737


In [13]:
res10.text

'<!DOCTYPE html>\n<html dir="ltr">\n<head>\n\t<meta content="text/html; charset=utf-8" http-equiv="content-type">\n\t<meta content="IE=edge" http-equiv="X-UA-Compatible"><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var o=t[n]={exports:{}};e[n][0].call(o.exports,function(t){var o=e[n][1][t];return r(o||t)},o,o.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(e,t,n){function r(){}function o(e,t,n){return function(){return i(e,[f.now()].concat(u(arguments)),t?null:this,n),t?void 0:this}}var i=e("handle"),a=e(2),u=e(3),c=e("ee").get("tracer"),f=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn-";a(p,function(e,t){s[t]=o(d+t,!0,"api")}),s.addPageAction=o(d+"addPageAction",!0),s.setCurren